<a href="https://colab.research.google.com/github/ramizcihe/week5-cihe240058/blob/main/ramiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required packages
!pip install pandas numpy --quiet

import pandas as pd
import numpy as np
import random

# -------------------------------
# Define labels
# -------------------------------
labels = [
    "not_cyberbullying",
    "ethnicity",
    "gender",
    "age",
    "religion",
    "appearance",
    "other_cyberbullying"
]

# Example text templates for each label
examples = {
    "not_cyberbullying": [
        "Good morning everyone! Stay positive today 😊",
        "Congrats on your promotion, well deserved!",
        "That movie was amazing, I highly recommend it.",
        "I love this new song, it's so catchy.",
        "Have a great day ahead!"
    ],
    "ethnicity": [
        "Go back to your country, you don’t belong here.",
        "People like you ruin this nation.",
        "You speak funny, learn proper English.",
        "No one wants your kind here.",
        "Stop acting superior just because of your race."
    ],
    "gender": [
        "Women can’t play this game, go to the kitchen.",
        "You throw like a girl, pathetic.",
        "Men are always stronger, you’re weak.",
        "No guy would ever want you.",
        "Girls don’t belong in tech."
    ],
    "age": [
        "You’re too old for this app, grandpa.",
        "Kids these days are so dumb.",
        "Nobody listens to old people anymore.",
        "Go play with your toys, child.",
        "You’re ancient, stop embarrassing yourself."
    ],
    "religion": [
        "Your religion is a joke.",
        "People like you should not be allowed here.",
        "Stop spreading your beliefs, nobody cares.",
        "We don’t want your religious nonsense.",
        "Your faith makes no sense at all."
    ],
    "appearance": [
        "You’re so ugly, nobody likes you.",
        "Look at your face, disgusting.",
        "You’re fat and gross.",
        "Your style is trash.",
        "No wonder you’re alone, you’re hideous."
    ],
    "other_cyberbullying": [
        "You play like trash, no skills at all.",
        "You’re the dumbest person I’ve ever met.",
        "Nobody cares about your opinion.",
        "Stop posting nonsense, nobody likes you.",
        "You’re such a loser, go away."
    ]
}

# -------------------------------
# Generate 1000 rows
# -------------------------------
data = []
for i in range(1, 1001):
    label = random.choice(labels)
    text = random.choice(examples[label])
    binary_label = "not_cyberbullying" if label == "not_cyberbullying" else "bullying"
    data.append([i, text, label, binary_label])

# Create DataFrame
df = pd.DataFrame(data, columns=["id", "text", "label", "binary_label"])

# Save to CSV
csv_path = "/content/cyberbullying_dataset.csv"
df.to_csv(csv_path, index=False)
print("✅ Dataset created and saved:", csv_path)
df.head()


✅ Dataset created and saved: /content/cyberbullying_dataset.csv


,id,text,label,binary_label
0,1,"Look at your face, disgusting.",appearance,bullying
1,2,Your faith makes no sense at all.,religion,bullying
2,3,Stop acting superior just because of your race.,ethnicity,bullying
3,4,Have a great day ahead!,not_cyberbullying,not_cyberbullying
4,5,Nobody cares about your opinion.,other_cyberbullying,bullying


In [ ]:
from google.colab import files
files.download("/content/cyberbullying_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.to_csv("https://raw.githubusercontent.com/ramizcihe/week5-cihe240058/refs/heads/main/cyberbullying_dataset.csv", index=False)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df[["text", "binary_label"]],
    test_size=0.2,
    random_state=42,
    stratify=df["binary_label"]
)


In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


In [ ]:
from torch.utils.data import Dataset
import torch

class CyberDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(int(self.labels[idx]), dtype=torch.long)
        return item


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np

# Ensure binary labels are numeric
train_df["binary_label"] = train_df["binary_label"].replace({
    "not_cyberbullying": 0,
    "bullying": 1
})

# Compute class weights
cw = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_df["binary_label"].values),
    y=train_df["binary_label"].values
)

# Convert to tensor for PyTorch
class_weights_tensor = torch.tensor(cw, dtype=torch.float)
print("Class Weights:", class_weights_tensor)


Class Weights: tensor([3.2520, 0.5908])


/tmp/ipython-input-1710137187.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df["binary_label"] = train_df["binary_label"].replace({


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments
from torch import nn

class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        self.loss_fn = nn.CrossEntropyLoss(weight=self.class_weights.to(self.model.device)) if class_weights is not None else nn.CrossEntropyLoss()

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [ ]:
import evaluate as hf_evaluate
metric_acc = hf_evaluate.load("accuracy")
metric_f1  = hf_evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1": metric_f1.compute(predictions=preds, references=labels, average="binary")["f1"]
    }


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,                # you can increase later
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",             # <-- FIXED for your version
    save_strategy="epoch",             # save checkpoints each epoch
    load_best_model_at_end=True
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from transformers import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor.to(logits.device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
# K-Fold Cross-Validation
from transformers import Trainer
import torch
from torch import nn

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fn = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device) if self.class_weights is not None else None)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [ ]:
# K-Fold loop:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    class_weights=class_weights_tensor   # Now works
)


In [ ]:
# ===============================
# Step 0: Install & Imports
# ===============================
!pip install transformers datasets evaluate scikit-learn --quiet

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch import nn
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate as hf_evaluate
import os

# Disable wandb to avoid login issues
os.environ["WANDB_DISABLED"] = "true"

# ===============================
# Step 1: Load Dataset
# ===============================
# Replace this with your CSV path if needed
CSV_PATH = "https://raw.githubusercontent.com/ramizcihe/week5-cihe240058/refs/heads/main/cyberbullying_dataset.csv"
df = pd.read_csv("https://raw.githubusercontent.com/ramizcihe/week5-cihe240058/refs/heads/main/cyberbullying_dataset.csv")

# Map binary labels to integers
df["binary_label"] = df["binary_label"].replace({
    "not_cyberbullying": 0,
    "bullying": 1
})

print("Sample Dataset:")
print(df.head())

# ===============================
# Step 2: Tokenizer
# ===============================
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ===============================
# Step 3: Dataset Class
# ===============================
class CyberDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(int(self.labels[idx]), dtype=torch.long)
        return item

# ===============================
# Step 4: Metrics
# ===============================
metric_acc = hf_evaluate.load("accuracy")
metric_f1  = hf_evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1": metric_f1.compute(predictions=preds, references=labels, average="binary")["f1"]
    }

# ===============================
# Step 5: Training Arguments
# ===============================
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",   # For older versions of Transformers
    save_strategy="epoch",
    load_best_model_at_end=True
)

# ===============================
# Step 6: Custom Weighted Trainer
# ===============================
class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fn = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device) if self.class_weights is not None else None)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# ===============================
# Step 7: K-Fold Cross-Validation
# ===============================
print("\n===== Starting 5-Fold Stratified Cross-Validation =====\n")

texts = df["text"].tolist()
labels = df["binary_label"].tolist()
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

fold = 1
acc_scores = []
f1_scores = []

for train_index, val_index in skf.split(texts, labels):
    print(f"\n----- Fold {fold} -----")

    X_train = [texts[i] for i in train_index]
    y_train = [labels[i] for i in train_index]
    X_val   = [texts[i] for i in val_index]
    y_val   = [labels[i] for i in val_index]

    # Create datasets
    train_ds = CyberDataset(X_train, y_train, tokenizer)
    val_ds   = CyberDataset(X_val, y_val, tokenizer)

    # Compute class weights
    cw = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(y_train),
        y=y_train
    )
    class_weights_tensor = torch.tensor(cw, dtype=torch.float)
    print("Class weights for this fold:", class_weights_tensor)

    # Initialize model
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

    # Initialize trainer
    trainer = WeightedTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
        class_weights=class_weights_tensor
    )

    # Train
    trainer.train()

    # Evaluate
    metrics = trainer.evaluate()
    print(f"Fold {fold} Accuracy: {metrics['eval_accuracy']:.4f}, F1: {metrics['eval_f1']:.4f}")
    acc_scores.append(metrics["eval_accuracy"])
    f1_scores.append(metrics["eval_f1"])

    fold += 1

print("\n===== K-Fold Cross-Validation Completed =====")
print("Average Accuracy:", np.mean(acc_scores))
print("Average F1-score:", np.mean(f1_scores))


/tmp/ipython-input-1723573604.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["binary_label"] = df["binary_label"].replace({


Sample Dataset:
   id                                             text                label  \
0   1                   Look at your face, disgusting.           appearance   
1   2                Your faith makes no sense at all.             religion   
2   3  Stop acting superior just because of your race.            ethnicity   
3   4                          Have a great day ahead!    not_cyberbullying   
4   5                 Nobody cares about your opinion.  other_cyberbullying   

   binary_label  
0             1  
1             1  
2             1  
3             0  
4             1  


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



===== Starting 5-Fold Stratified Cross-Validation =====


----- Fold 1 -----
Class weights for this fold: tensor([3.2258, 0.5917])


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

Fold 4 Accuracy: 1.0000, F1: 1.0000

----- Fold 5 -----
Class weights for this fold: tensor([3.2520, 0.5908])


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.631500,0.581653,1.000000,1.000000
2,0.102400,0.051777,1.000000,1.000000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Fold 5 Accuracy: 1.0000, F1: 1.0000

===== K-Fold Cross-Validation Completed =====
Average Accuracy: 1.0
Average F1-score: 1.0


In [ ]:
# ===============================
# Step 12 & 13: Feature Enhancements + Evaluation
# ===============================

# --- Install Required Packages ---
!pip install -q textblob wordcloud transformers

# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, accuracy_score, f1_score
from wordcloud import WordCloud
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch.nn as nn

# --- Load Dataset ---
df = pd.read_csv("https://raw.githubusercontent.com/ramizcihe/week5-cihe240058/refs/heads/main/cyberbullying_dataset.csv")

# Use correct columns
df.dropna(subset=["text"], inplace=True)
df["binary_label"] = df["label"].apply(lambda x: 0 if x == "not_cyberbullying" else 1)

print("✅ Dataset loaded. Sample rows:")
print(df.head())

# --- Step 12: Feature Enhancements ---
# Sentiment polarity
df["sentiment"] = df["text"].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# N-gram features (for visualization only)
vectorizer = CountVectorizer(ngram_range=(1,2), max_features=100)
ngram_matrix = vectorizer.fit_transform(df["text"])

print("\n✅ Sentiment & n-gram features added.")
print(df[["text", "binary_label", "sentiment"]].head())

# --- Step 13: Evaluation & Visualization ---
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Dataset wrapper
from torch.utils.data import Dataset
class CyberDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

final_train_ds = CyberDataset(df["text"].tolist(), df["binary_label"].tolist(), tokenizer)

# Compute class weights
cw = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(df["binary_label"].values),
    y=df["binary_label"].values
)
class_weights_tensor = torch.tensor(cw, dtype=torch.float32)

# Custom Weighted Trainer
class WeightedTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",   # ✅ fixed argument
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

# Initialize model
final_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

# Initialize Trainer
final_trainer = WeightedTrainer(
    model=final_model,
    args=training_args,
    train_dataset=final_train_ds,
    eval_dataset=final_train_ds,
    compute_metrics=compute_metrics,
    class_weights=class_weights_tensor
)

# Train
final_trainer.train()

# Predict
preds_output = final_trainer.predict(final_train_ds)
preds = np.argmax(preds_output.predictions, axis=1)
labels_true = df["binary_label"].values

# --- Visualizations ---

# 1️⃣ Confusion Matrix
cm = confusion_matrix(labels_true, preds)
plt.figure(figsize=(5,4))
plt.title("Confusion Matrix")
plt.imshow(cm, cmap=plt.cm.Blues)
plt.colorbar()
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# 2️⃣ ROC Curve
fpr, tpr, _ = roc_curve(labels_true, preds_output.predictions[:,1])
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0,1],[0,1], color="navy", lw=2, linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC)")
plt.legend(loc="lower right")
plt.show()

# 3️⃣ Precision-Recall Curve
precision, recall, _ = precision_recall_curve(labels_true, preds_output.predictions[:,1])
plt.figure()
plt.plot(recall, precision, lw=2, color="purple")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.show()

# 4️⃣ Word Cloud of Misclassified Tweets
misclassified_texts = df["text"][labels_true != preds]
if len(misclassified_texts) > 0:
    text_combined = " ".join(misclassified_texts)
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text_combined)
    plt.figure(figsize=(10,5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("Word Cloud of Misclassified Tweets")
    plt.show()
else:
    print("✅ No misclassified tweets found — model classified all correctly!")


✅ Dataset loaded. Sample rows:
   id                                             text                label  \
0   1                   Look at your face, disgusting.           appearance   
1   2                Your faith makes no sense at all.             religion   
2   3  Stop acting superior just because of your race.            ethnicity   
3   4                          Have a great day ahead!    not_cyberbullying   
4   5                 Nobody cares about your opinion.  other_cyberbullying   

   binary_label  
0             1  
1             1  
2             1  
3             0  
4             1  

✅ Sentiment & n-gram features added.
                                              text  binary_label  sentiment
0                   Look at your face, disgusting.             1      -1.00
1                Your faith makes no sense at all.             1       0.00
2  Stop acting superior just because of your race.             1       0.35
3                          Have a great day 

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: